<a href="https://colab.research.google.com/github/vinay-852/Video_Synopsis/blob/main/Video_Synopsis(Clear).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Copy data without downloading.......
import os
import requests
import tqdm

def text_retriever(url) -> list[str]:
  """
  Retrieves URLs from a text file.

  Args:
      url (str): The URL of the text file containing video URLs.

  Returns:
      list[str]: A list of video URLs extracted from the text file.

  Raises:
      Exception: If there's an error retrieving or parsing the text file.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for non-2xx status codes

    text = response.text
    urls = text.splitlines()  # Split text into lines, removing unnecessary spaces

    return urls
  except Exception as e:
    print(f"Error retrieving URLs: {e}")
    return []

In [2]:
urls=text_retriever("https://raw.githubusercontent.com/Kitware/MEVID/refs/heads/main/mevid-v1-video-URLS.txt")

In [3]:
#Download Dataset form internet.........
import os
import requests
from tqdm import tqdm

def download_file(url, save_dir):
  """
  Download a file from a given URL and save it to a specified directory.

  Args:
    url: The URL of the file to download.
    save_dir: The directory where the file should be saved.

  Returns:
    None
  """
  try:
      if not os.path.exists(save_dir):
          os.makedirs(save_dir)

      filename = url.split('/')[-1]
      file_path = os.path.join(save_dir, filename)

      response = requests.get(url, stream=True)
      total_size = int(response.headers.get('content-length', 0))

      with open(file_path, 'wb') as f:
          with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading') as progress_bar:
              for data in response.iter_content(chunk_size=1024):
                  if data:
                      progress_bar.update(len(data))
                      f.write(data)

      print(f"Download complete! File saved to: {file_path}")
  except Exception as e:
      print(f"Error downloading file: {e}")

In [4]:
download_file(urls[0], "/content/dataset")

Downloading: 100%|██████████| 162M/162M [00:18<00:00, 8.71MB/s]

Download complete! File saved to: /content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi


In [5]:
!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.1 MB/s eta 0:00:00


In [6]:
download_file("https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt","/content/weights")

Downloading: 100%|██████████| 104M/104M [00:13<00:00, 7.99MB/s]

Download complete! File saved to: /content/weights/yolov10l.pt


In [7]:
#Video Format which can be played in Colab..............
from IPython.display import HTML
from base64 import b64encode
def play_video(filename):
  html = ''
  video = open(filename,'rb').read()
  src = 'data:video/mp4;base64,' + b64encode(video).decode()
  html += fr'<video width=900 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
  return HTML(html)

In [8]:
#Display on Colab ........
from IPython.display import HTML
from base64 import b64encode
import subprocess

def process_and_play_video(input_filename, output_filename):
  """
  Uses the ffmpeg command to process a video file and play it in Colab.

  Args:
      input_filename (str): The path to the input video file.
      output_filename (str): The path to save the processed video file.

  Returns:
      HTML: An HTML element to display the processed video in Colab.
  """
  subprocess.run([
        'ffmpeg',
        '-hide_banner',
        '-loglevel', 'error',
        '-i', input_filename,
        '-vcodec', 'libx264',
        output_filename,
        '-y'
    ])

  return play_video(output_filename)

In [9]:
import cv2
import datetime
from absl import app, flags
from absl.flags import FLAGS
from deep_sort_realtime.deepsort_tracker import DeepSort
import os
import csv
import numpy as np
from typing import Tuple, List
import torch
from tqdm import tqdm

In [10]:
#YOLOv10 model Operations..........
from typing import Tuple, List
import numpy as np
from ultralytics import YOLOv10

# Load YOLO model and COCO classes
def load_yolo_model(model_file, class_file="coco.names"):
    """
    Loads the YOLO model for object detection and the class names from the COCO dataset.

    Args:
        model_file (str): Path to the YOLO model file.
        class_file (str): Path to the file containing COCO class names.

    Returns:
        tuple:
            - model (YOLOv10): The loaded YOLO model.
            - class_names (list): List of class names corresponding to COCO dataset classes.
    """
    # Check if a GPU is available; if not, use the CPU
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    print(f'Using device: {device}')

    # Load the YOLO model
    model = YOLOv10(model_file)

    # Load COCO class names from the specified file
    class_names=text_retriever("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names")

    return model, class_names

def detect_objects(model, frame, conf_threshold):
    """
    Performs object detection on a given frame using the YOLO model.

    Args:
        model (YOLOv10): The loaded YOLO model used for detecting objects.
        frame (numpy.ndarray): The input video frame to perform object detection on.
        conf_threshold (float): Confidence threshold to filter weak detections.

    Returns:
        list: A list of detections where each detection contains bounding box, confidence, and class ID.
    """
    # Perform object detection on the frame
    results = model(frame, verbose=False)[0]

    detections = []
    for det in results.boxes:
        confidence = det.conf
        label = det.cls
        bbox = det.xyxy[0]  # Bounding box coordinates: [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(int, bbox)
        class_id = int(label)

        # Filter out weak detections based on confidence threshold
        if confidence >= conf_threshold:
            detections.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    return detections

In [11]:
download_file("https://i.cdn.newsbytesapp.com/images/l71620241008162609.jpeg", "/content/")

Downloading: 100%|██████████| 132k/132k [00:00<00:00, 3.19MB/s]

Download complete! File saved to: /content/l71620241008162609.jpeg


In [12]:
model, class_names =load_yolo_model("/content/weights/yolov10l.pt")

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


Using device: cpu


In [13]:
detections = detect_objects(model, '/content/l71620241008162609.jpeg', conf_threshold=0.5)

In [14]:
print(detections)

[[[886, 78, 274, 968], tensor([0.9338]), 0], [[1477, 433, 136, 341], tensor([0.8929]), 0], [[941, 19, 306, 358], tensor([0.8528]), 34], [[1649, 543, 53, 138], tensor([0.8219]), 0], [[1745, 562, 47, 102], tensor([0.7588]), 0], [[1829, 554, 47, 122], tensor([0.7557]), 0], [[1874, 544, 45, 135], tensor([0.7498]), 0], [[1465, 526, 48, 53], tensor([0.7447]), 35], [[736, 583, 44, 73], tensor([0.5945]), 0], [[434, 573, 37, 82], tensor([0.5820]), 0], [[815, 592, 38, 64], tensor([0.5511]), 0]]


In [15]:
import cv2
import numpy as np

def generate_background(video_file, output_file='background.jpg', method='median', num_frames=30):
    """
    Generate a background image from the first 'num_frames' frames of a video by averaging or taking the median.

    Args:
        video_file (str): Path to the input video file.
        output_file (str): Path where the background image will be saved.
        method (str): Method to generate background ('mean' or 'median').
        num_frames (int): Number of frames to use for background generation.

    Returns:
        None
    """
    # Open the video file
    cap = cv2.VideoCapture(video_file)

    # Check if the video was opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Initialize a list to store frames
    frames = []
    frame_count = 0

    # Read frames from the video until num_frames are collected
    while frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"End of video reached at frame {frame_count}.")
            break

        # Append the frame to the list
        frames.append(frame)
        frame_count += 1

    # Release the video capture object
    cap.release()

    # Convert the list of frames to a numpy array
    frames = np.array(frames)

    # Generate the background by computing the median or mean of the frames
    if method == 'median':
        background = np.median(frames, axis=0).astype(dtype=np.uint8)  # Median
    elif method == 'mean':
        background = np.mean(frames, axis=0).astype(dtype=np.uint8)  # Mean
    else:
        print("Error: Method must be 'mean' or 'median'.")
        return

    # Save the generated background as an image file
    cv2.imwrite(output_file, background)
    print(f"Background image saved as {output_file}")

# Example usage:
video_file = '/content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi'  # Replace with the path to your video file
generate_background(video_file, output_file='background.jpg', method='median', num_frames=30)


Background image saved as background.jpg


In [16]:
#Deepsort Operations...........
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

def initialize_deepsort(max_age=20, n_init=3):
    """
    Initializes the DeepSort tracker.

    Args:
        max_age (int): Maximum number of frames to keep alive a track.
        n_init (int): Minimum number of detections before a track is confirmed.

    Returns:
        DeepSort: An instance of the DeepSort tracker.
    """
    return DeepSort(max_age=max_age, n_init=n_init)

def update_tracker(tracker, detections, frame):
    """
    Updates the DeepSort tracker with new detections.

    Args:
        tracker (DeepSort): The DeepSort tracker instance.
        detections (list): List of detections in the format [bounding_box, confidence, class_id].
        frame (numpy.ndarray): The current video frame.

    Returns:
        List: Updated tracks from the DeepSort tracker.
    """
    return tracker.update_tracks(detections, frame=frame)


In [17]:
#Useful Utility function to complete tracking video.........
import os
import cv2
import csv
import numpy as np
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
from tqdm import tqdm

# Initialize video capture and writer
def initialize_video(video_file, output_file):
    """
    Initializes video capture and writer objects for reading from the input video and writing to the output video.

    Args:
        video_file (str): Path to the input video file.
        output_file (str): Path to the output video file where the processed video will be saved.

    Returns:
        tuple:
            - video_cap (cv2.VideoCapture): The video capture object.
            - writer (cv2.VideoWriter): The video writer object for saving processed frames.
            - total_frames (int): Total number of frames in the input video.
            - frame_width (int): Width of each frame in the video.
            - frame_height (int): Height of each frame in the video.
    """
    video_cap = cv2.VideoCapture(video_file)
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    total_frames = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frames for tqdm
    return video_cap, writer, total_frames, frame_width, frame_height


def extract_hsv(frame, ltrb):
    """
    Extracts the mean hue, saturation, and value (HSV) from the region of interest (ROI) defined by the bounding box.

    Args:
        frame (numpy.ndarray): The current video frame.
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2) defining the ROI.

    Returns:
        tuple:
            - mean_hue (float): Mean hue value in the ROI.
            - mean_saturation (float): Mean saturation value in the ROI.
            - mean_value (float): Mean value (brightness) in the ROI.
    """
    x1, y1, x2, y2 = map(int, ltrb)
    # Ensure the bounding box coordinates are within the frame boundaries
    x1, y1, x2, y2 = max(0, x1), max(0, y1), min(frame.shape[1], x2), min(frame.shape[0], y2)

    if x2 > x1 and y2 > y1:
        # Extract the region of interest (ROI) based on the bounding box
        object_roi = frame[y1:y2, x1:x2]

        # Convert the ROI to HSV color space
        hsv_roi = cv2.cvtColor(object_roi, cv2.COLOR_BGR2HSV)

        # Compute the mean values for hue, saturation, and value in the HSV space
        mean_hue = np.mean(hsv_roi[:, :, 0])
        mean_saturation = np.mean(hsv_roi[:, :, 1])
        mean_value = np.mean(hsv_roi[:, :, 2])

        return mean_hue, mean_saturation, mean_value
    return None, None, None

# Write tracking data to CSV
def write_csv(writer_csv, track_id, class_name, frame_count, bbox, hue, saturation, value):
    """
    Writes the tracking data for an object to the CSV file, including track ID, class, frame number, bounding box, and HSV color information.

    Args:
        writer_csv (csv.writer): CSV writer object to save tracking data.
        track_id (int): Unique ID for the tracked object.
        class_name (str): Name of the detected object class.
        frame_count (int): Current frame number.
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2).
        hue (float): Mean hue value of the object in the ROI.
        saturation (float): Mean saturation value of the object in the ROI.
        value (float): Mean value (brightness) of the object in the ROI.
    """
    writer_csv.writerow([track_id, class_name, frame_count, bbox, hue, saturation, value])

In [18]:
import csv
import cv2
import numpy as np
from tqdm import tqdm

def calculate_frame_difference(prev_frame, curr_frame):
    """
    Calculate the difference between two frames using Mean Squared Error (MSE).

    Args:
        prev_frame (np.array): The previous frame.
        curr_frame (np.array): The current frame.

    Returns:
        float: The difference between the two frames.
    """
    # Convert frames to grayscale for simplicity
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Compute Mean Squared Error (MSE) between the two frames
    diff = np.sum((prev_gray.astype("float") - curr_gray.astype("float")) ** 2)
    diff /= float(prev_gray.shape[0] * prev_gray.shape[1])

    return diff



In [ ]:
import os
import cv2
import csv
import numpy as np
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort

def extract_object_tracks(model_file, video_file, output_file, conf_threshold, threshold, crops_dir="object_crops"):
    """
    Main function to extract object tracks from key frames of a video using YOLO for object detection and
    DeepSort for tracking. The function saves the tracking data, including bounding boxes, HSV values, and
    cropped objects to a CSV file, and generates an output video with annotated tracks.

    Args:
        model_file (str): Path to the YOLO model file.
        video_file (str): Path to the input video file.
        output_file (str): Path to the output video file to save the annotated video.
        conf_threshold (float): Confidence threshold to filter weak detections.
        threshold (float): Threshold for selecting key frames based on frame difference.
        crops_dir (str): Directory where cropped object images will be saved.
    """
    try:
        # Initialize video capture, writer, and other video properties
        video_cap, writer, total_frames, frame_width, frame_height = initialize_video(video_file, output_file)

        # Load YOLO model and COCO class names
        model, class_names = load_yolo_model(model_file)

        # Initialize DeepSort tracker
        tracker = DeepSort(max_age=20, n_init=3)

        # Create directory for saving crops if it doesn't exist
        if not os.path.exists(crops_dir):
            os.makedirs(crops_dir)

        # Open CSV file for saving object tracks
        with open('object_tracks.csv', mode='w', newline='') as file:
            writer_csv = csv.writer(file)
            writer_csv.writerow(['Track ID', 'Class Name', 'Frame', 'Bounding Box', 'Hue', 'Saturation', 'Value', 'Crop Path'])

            frame_count = 0
            prev_frame = None

            # Progress bar for processing video frames
            with tqdm(total=total_frames, desc="Processing Frames") as pbar:
                while True:
                    ret, frame = video_cap.read()
                    frame_count += 1
                    if not ret:
                        print("End of video.")
                        break

                    # If this is the first frame, store it and continue to the next
                    if prev_frame is None:
                        prev_frame = frame
                        continue

                    # Calculate the difference between the current frame and the previous frame
                    frame_diff = calculate_frame_difference(prev_frame, frame)

                    # Check if the difference exceeds the threshold (key frame selection)
                    if frame_diff > threshold:
                        print(f"Processing key frame {frame_count} (difference: {frame_diff})")

                        # Object detection
                        detections = detect_objects(model, frame, conf_threshold)
                        tracks = update_tracker(tracker, detections, frame)

                        for track in tracks:
                            if track.is_confirmed():
                                track_id = track.track_id
                                ltrb = track.to_ltrb()  # Bounding box format: (left, top, right, bottom)
                                class_id = track.get_det_class()
                                class_name = class_names[class_id]

                                # Extract HSV color features from the bounding box region
                                hue, saturation, value = extract_hsv(frame, ltrb)

                                # Crop the object and save the cropped image
                                x1, y1, x2, y2 = map(int, ltrb)
                                crop_img = frame[y1:y2, x1:x2]
                                crop_path = f"{crops_dir}/track_{track_id}_frame_{frame_count}.png"
                                cv2.imwrite(crop_path, crop_img)

                                if hue is not None:
                                    # Write the tracking data (including HSV and crop path) to the CSV file
                                    writer_csv.writerow([track_id, class_name, frame_count, ltrb, hue, saturation, value, crop_path])

                                    # Draw bounding box and label on the frame
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                    cv2.putText(frame, f'{class_name} {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                        # Write the frame with annotations to the output video
                        writer.write(frame)

                    # Update the previous frame
                    prev_frame = frame
                    pbar.update(1)

        # Release video capture and writer resources
        video_cap.release()
        writer.release()
        print(f"Tracking completed and saved to 'object_tracks.csv'. Cropped images saved to '{crops_dir}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage of the function with keyframe selection and cropping
model_file = "weights/yolov10l.pt"
video_file = "/content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi"
output_file = "output_tracked_keyframes.mp4"
conf_threshold = 0.5
frame_diff_threshold = 6  # Set a threshold for keyframe selection

# Call the main function to extract object tracks based on key frames and crop objects
extract_object_tracks(model_file, video_file, output_file, conf_threshold, frame_diff_threshold)


Using device: cpu


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
/usr/local/lib

Processing key frame 61 (difference: 12.92500048585199)
Processing key frame 62 (difference: 16.59521047108209)


Processing Frames:   1%|▏         | 118/9001 [00:10<05:14, 28.23it/s]

Processing key frame 121 (difference: 15.31318116449005)
Processing key frame 122 (difference: 16.554047632929105)


...

Processing Frames:   7%|▋         | 597/9001 [02:57<4:38:52,  1.99s/it]

Processing key frame 599 (difference: 27.734794290267413)


Processing Frames:   7%|▋         | 598/9001 [02:59<4:21:52,  1.87s/it]

Processing key frame 600 (difference: 26.48828076414801)


Processing Frames:   7%|▋         | 599/9001 [03:00<4:05:38,  1.75s/it]

In [ ]:
import csv
import math

def calculate_area_velocity_tubes(input_csv, output_csv):
    """
    Calculate area and velocity for each object in the object_tracks.csv file by considering objects in the same tube
    (i.e., same Track ID) and computing the velocity based on the movement between consecutive frames.

    Args:
        input_csv (str): Path to the input CSV file (object_tracks.csv).
        output_csv (str): Path to the output CSV file to save the results with area and velocity.
    """
    try:
        rows = []

        # Read the input CSV file
        with open(input_csv, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Convert frame to int and bounding box values to float for calculations
                row['Frame'] = int(row['Frame'])

                # Parse the bounding box values by removing brackets and splitting by commas or spaces
                bbox_str = row['Bounding Box'].strip('[]')
                row['Bounding Box'] = tuple(map(float, bbox_str.replace(",", " ").split()))

                rows.append(row)

        # Sort rows by Track ID and Frame to ensure sequential processing
        rows.sort(key=lambda x: (int(x['Track ID']), x['Frame']))

        # Open the output CSV file for writing
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            # Write the new header with Area and Velocity columns
            writer.writerow(['Track ID', 'Class Name', 'Frame', 'Bounding Box', 'Hue', 'Saturation', 'Value', 'Area', 'Velocity'])

            prev_objects = {}  # To store the last seen object positions per Track ID

            # Loop through the rows and calculate area and velocity
            for row in rows:
                try:
                    # Extract bounding box values
                    left, top, right, bottom = row['Bounding Box']

                    # Calculate area of the bounding box
                    area = (right - left) * (bottom - top)

                    # Calculate velocity for the same Track ID between consecutive frames
                    track_id = row['Track ID']
                    current_frame = row['Frame']

                    # Calculate center of the current bounding box
                    center_x = (left + right) / 2
                    center_y = (top + bottom) / 2

                    # Check if we have a previous record for the same track_id (same tube)
                    if track_id in prev_objects:
                        prev_frame, (prev_center_x, prev_center_y) = prev_objects[track_id]

                        # Calculate velocity if the object exists in the previous frame (adjacent frames only)
                        if current_frame == prev_frame + 1:
                            velocity = math.sqrt((center_x - prev_center_x) ** 2 + (center_y - prev_center_y) ** 2)
                        else:
                            velocity = 0  # If not consecutive frames, velocity is 0
                    else:
                        velocity = 0  # First appearance of the object, velocity is 0

                    # Update the previous object record with the current frame and center
                    prev_objects[track_id] = (current_frame, (center_x, center_y))

                    # Write the row to the new CSV file with calculated area and velocity
                    writer.writerow([
                        row['Track ID'], row['Class Name'], row['Frame'], row['Bounding Box'],
                        row['Hue'], row['Saturation'], row['Value'], area, velocity
                    ])

                except Exception as row_error:
                    print(f"Error processing row: {row}, error: {row_error}")
                    continue  # Skip any problematic rows

        print(f"Area and velocity calculations completed and saved to {output_csv}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function with the input and output CSV file paths
input_csv = 'object_tracks.csv'
output_csv = 'object_tracks_with_area_velocity_tubes.csv'

calculate_area_velocity_tubes(input_csv, output_csv)


In [ ]:
import pandas as pd

df = pd.read_csv('object_tracks_with_area_velocity_tubes.csv')
df.head(50)

In [ ]:
import numpy as np
import pandas as pd
import random
import math
from copy import deepcopy

# Load database from CSV file with additional object information
def load_tracking_database(csv_file):
    df = pd.read_csv(csv_file)

    # Clean up the 'Bounding Box' column and convert to list of coordinates
    df['Bounding Box'] = df['Bounding Box'].apply(lambda x: [int(coord) for coord in x.strip('()').split(',')])

    # Filter to only include persons
    df = df[df['Class Name'] == 'person']

    # Convert dataframe to list of dictionaries
    database = df.to_dict(orient='records')
    return database

# Group the tracking data by Track ID
def group_tracks_by_id(database):
    grouped_tracks = {}
    for record in database:
        track_id = record['Track ID']
        if track_id not in grouped_tracks:
            grouped_tracks[track_id] = []
        grouped_tracks[track_id].append(record)
    return list(grouped_tracks.values())

# Utility to compute Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_b, y1_b, x2_b, y2_b = box2

    # Calculate intersection
    inter_x1 = max(x1, x1_b)
    inter_y1 = max(y1, y1_b)
    inter_x2 = min(x2, x2_b)
    inter_y2 = min(y2, y2_b)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_b - x1_b) * (y2_b - y1_b)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Temporal coherence: smoothness of bounding box motion over frames
def temporal_coherence(tracks):
    coherence_score = 0
    for track in tracks:
        for i in range(1, len(track)):
            box_prev = track[i-1]['Bounding Box']
            box_curr = track[i]['Bounding Box']
            displacement = np.linalg.norm(np.array(box_prev[:2]) - np.array(box_curr[:2]))
            coherence_score += np.exp(-displacement)
    return coherence_score

# Spatial coherence: minimize overlap between bounding boxes in the same frame
def spatial_coherence(tracks):
    overlap_penalty = 0
    frames = {}
    for track in tracks:
        for frame_info in track:
            frame = frame_info['Frame']
            box = frame_info['Bounding Box']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(box)

    for boxes in frames.values():
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                overlap_penalty += max(0, iou(boxes[i], boxes[j]) - 0.1)  # Penalize for high overlap

    return overlap_penalty

# Fitness function with temporal and spatial coherence
def fitness(tracks, w1=1.5, w2=2):
    return w1 * temporal_coherence(tracks) - w2 * spatial_coherence(tracks)

# Scale down the bounding box to reduce chances of overlap
def scale_bbox(bbox, scale_factor=0.5):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    new_width = width * scale_factor
    new_height = height * scale_factor

    new_x1 = center_x - new_width / 2
    new_y1 = center_y - new_height / 2
    new_x2 = center_x + new_width / 2
    new_y2 = center_y + new_height / 2

    return [new_x1, new_y1, new_x2, new_y2]

# Shift bounding boxes slightly to reduce overlap
def shift_bbox(bbox, shift_value=2):
    x1, y1, x2, y2 = bbox
    return [x1 + shift_value, y1 + shift_value, x2 + shift_value, y2 + shift_value]

# Adjust BBoxes in a frame to minimize overlap
def adjust_frame_bboxes(frame_boxes):
    adjusted_boxes = []
    for i in range(len(frame_boxes)):
        for j in range(i + 1, len(frame_boxes)):
            if iou(frame_boxes[i], frame_boxes[j]) > 0:
                # If overlapping, apply a shift
                frame_boxes[j] = shift_bbox(frame_boxes[j])
        adjusted_boxes.append(scale_bbox(frame_boxes[i]))
    return adjusted_boxes

# Mutate by scaling and shifting bounding boxes to avoid overlaps
def mutate(individual, mutation_rate=0.05, max_shift=3, max_bbox_shift=3):
    mutated = deepcopy(individual)
    frames = {}

    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(obj['Bounding Box'])

    # Adjust BBoxes in each frame to avoid overlap
    for frame, boxes in frames.items():
        frames[frame] = adjust_frame_bboxes(boxes)

    # Apply adjusted BBoxes back to tracks
    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            obj['Bounding Box'] = frames[frame].pop(0)

    return mutated

# Simulated Annealing with track shifting and enhanced mutation
def simulated_annealing(database, initial_temperature=1000, cooling_rate=0.995, mutation_rate=0.1, max_generations=100):
    # Initialize population with a single solution
    current_solution = deepcopy(database)

    current_fitness = fitness(current_solution)
    temperature = initial_temperature

    for generation in range(max_generations):
        # Mutate the current solution
        new_solution = mutate(current_solution, mutation_rate)
        new_fitness = fitness(new_solution)

        # Calculate the acceptance probability
        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        # Cool down the temperature
        temperature *= cooling_rate
        if generation % 50 == 0 or generation == max_generations - 1:
            print(f"Generation {generation}, Fitness: {current_fitness:.4f}, Temperature: {temperature:.2f}")

    return current_solution

# Main script to load the CSV and run simulated annealing
if __name__ == "__main__":
    csv_file = '/content/object_tracks.csv'  # Replace with the path to your CSV file
    tracking_database = load_tracking_database(csv_file)

    # Group the tracks by Track ID
    grouped_tracks = group_tracks_by_id(tracking_database)

    # Run simulated annealing on the grouped tracks
    best_tracks = simulated_annealing(grouped_tracks)

    # Flatten the list of tracks for output
    optimized_data = []
    for track in best_tracks:
        optimized_data.extend(track)

    # Convert back to DataFrame for saving or further processing
    optimized_df = pd.DataFrame(optimized_data)
    optimized_df.to_csv('/content/optimized_person_tracks.csv', index=False)

    print("Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'")
    print(optimized_df.head())


In [ ]:
import numpy as np
import pandas as pd
import random
import math
from copy import deepcopy

# Load database from CSV file with additional object information
def load_tracking_database(csv_file):
    df = pd.read_csv(csv_file)

    # Clean up the 'Bounding Box' column and convert to list of coordinates
    df['Bounding Box'] = df['Bounding Box'].apply(lambda x: [int(coord) for coord in x.strip('[]').split(',')])

    # Filter to only include persons
    df = df[df['Class Name'] == 'person']

    # Convert dataframe to list of dictionaries
    database = df.to_dict(orient='records')
    return database

# Group the tracking data by Track ID
def group_tracks_by_id(database):
    grouped_tracks = {}
    for record in database:
        track_id = record['Track ID']
        if track_id not in grouped_tracks:
            grouped_tracks[track_id] = []
        grouped_tracks[track_id].append(record)
    return list(grouped_tracks.values())

# Utility to compute Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_b, y1_b, x2_b, y2_b = box2

    # Calculate intersection
    inter_x1 = max(x1, x1_b)
    inter_y1 = max(y1, y1_b)
    inter_x2 = min(x2, x2_b)
    inter_y2 = min(y2, y2_b)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_b - x1_b) * (y2_b - y1_b)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Temporal coherence: smoothness of bounding box motion over frames
def temporal_coherence(tracks):
    coherence_score = 0
    for track in tracks:
        for i in range(1, len(track)):
            box_prev = track[i-1]['Bounding Box']
            box_curr = track[i]['Bounding Box']
            displacement = np.linalg.norm(np.array(box_prev[:2]) - np.array(box_curr[:2]))
            coherence_score += np.exp(-displacement)
    return coherence_score

# Spatial coherence: minimize overlap between bounding boxes in the same frame
def spatial_coherence(tracks):
    overlap_penalty = 0
    frames = {}
    for track in tracks:
        for frame_info in track:
            frame = frame_info['Frame']
            box = frame_info['Bounding Box']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(box)

    for boxes in frames.values():
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                overlap_penalty += max(0, iou(boxes[i], boxes[j]) - 0.1)  # Penalize for high overlap

    return overlap_penalty

# Fitness function with temporal and spatial coherence
def fitness(tracks, w1=1.5, w2=2):
    return w1 * temporal_coherence(tracks) - w2 * spatial_coherence(tracks)

# Calculate area of a bounding box
def calculate_area(bbox):
    x1, y1, x2, y2 = bbox
    return (x2 - x1) * (y2 - y1)

# Calculate velocity based on consecutive frames for each track
def calculate_velocity(track):
    prev_frame, prev_center = None, None
    for obj in track:
        current_bbox = obj['Bounding Box']
        current_frame = obj['Frame']

        # Calculate center of the current bounding box
        center_x = (current_bbox[0] + current_bbox[2]) / 2
        center_y = (current_bbox[1] + current_bbox[3]) / 2

        if prev_frame is not None and current_frame == prev_frame + 1:
            # Calculate velocity if it's the next frame in the sequence
            velocity = math.sqrt((center_x - prev_center[0]) ** 2 + (center_y - prev_center[1]) ** 2)
        else:
            # First frame or non-consecutive frames
            velocity = 0

        # Store velocity and area in the object
        obj['Area'] = calculate_area(current_bbox)
        obj['Velocity'] = velocity

        # Update previous frame and center
        prev_frame = current_frame
        prev_center = (center_x, center_y)

    return track

# Scale down the bounding box to reduce chances of overlap
def scale_bbox(bbox, scale_factor=0.5):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    new_width = width * scale_factor
    new_height = height * scale_factor

    new_x1 = center_x - new_width / 2
    new_y1 = center_y - new_height / 2
    new_x2 = center_x + new_width / 2
    new_y2 = center_y + new_height / 2

    return [new_x1, new_y1, new_x2, new_y2]

# Shift bounding boxes slightly to reduce overlap
def shift_bbox(bbox, shift_value=2):
    x1, y1, x2, y2 = bbox
    return [x1 + shift_value, y1 + shift_value, x2 + shift_value, y2 + shift_value]

# Adjust BBoxes in a frame to minimize overlap
def adjust_frame_bboxes(frame_boxes):
    adjusted_boxes = []
    for i in range(len(frame_boxes)):
        for j in range(i + 1, len(frame_boxes)):
            if iou(frame_boxes[i], frame_boxes[j]) > 0:
                # If overlapping, apply a shift
                frame_boxes[j] = shift_bbox(frame_boxes[j])
        adjusted_boxes.append(scale_bbox(frame_boxes[i]))
    return adjusted_boxes

# Mutate by scaling and shifting bounding boxes to avoid overlaps
def mutate(individual, mutation_rate=0.05, max_shift=3, max_bbox_shift=3):
    mutated = deepcopy(individual)
    frames = {}

    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(obj['Bounding Box'])

    # Adjust BBoxes in each frame to avoid overlap
    for frame, boxes in frames.items():
        frames[frame] = adjust_frame_bboxes(boxes)

    # Apply adjusted BBoxes back to tracks
    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            obj['Bounding Box'] = frames[frame].pop(0)

    return mutated

# Simulated Annealing with track shifting and enhanced mutation
def simulated_annealing(database, initial_temperature=1000, cooling_rate=0.995, mutation_rate=0.1, max_generations=100):
    # Initialize population with a single solution
    current_solution = deepcopy(database)

    # Calculate area and velocity for the initial solution
    current_solution = [calculate_velocity(track) for track in current_solution]
    current_fitness = fitness(current_solution)
    temperature = initial_temperature

    for generation in range(max_generations):
        # Mutate the current solution
        new_solution = mutate(current_solution, mutation_rate)

        # Calculate area and velocity for the new solution
        new_solution = [calculate_velocity(track) for track in new_solution]
        new_fitness = fitness(new_solution)

        # Calculate the acceptance probability
        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        # Cool down the temperature
        temperature *= cooling_rate
        if generation % 50 == 0 or generation == max_generations - 1:
            print(f"Generation {generation}, Fitness: {current_fitness:.4f}, Temperature: {temperature:.2f}")

    return current_solution

# Main script to load the CSV and run simulated annealing
if __name__ == "__main__":
    csv_file = '/content/object_tracks.csv'  # Replace with the path to your CSV file
    tracking_database = load_tracking_database(csv_file)

    # Group the tracks by Track ID
    grouped_tracks = group_tracks_by_id(tracking_database)

    # Run simulated annealing on the grouped tracks
    best_tracks = simulated_annealing(grouped_tracks)

    # Flatten the list of tracks for output
    optimized_data = []
    for track in best_tracks:
        optimized_data.extend(track)

    # Convert back to DataFrame for saving or further processing
    optimized_df = pd.DataFrame(optimized_data)

    # Save optimized data with area and velocity included
    optimized_df.to_csv('/content/optimized_person_tracks1.csv', index=False)

    print("Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'")
    print(optimized_df.head())
